In [1]:
#!pip install timm

In [2]:
import pandas as pd
import timm
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
#from efficientnet_pytorch import EfficientNet
import numpy as np

from PIL import Image
from PIL import ImageFile

In [3]:
import pytorch_lightning
pytorch_lightning.seed_everything(42)

42

In [4]:
base_dir = r'C:\Users\Kaggle\Leaf_Classification\input'

In [5]:
### read csv and label mapping

labels = pd.read_csv(os.path.join(base_dir,'train.csv'))

with open(f'{base_dir}/label_num_to_disease_map.json') as f:
    label_mapper = json.load(f)

### Dataset class

In [6]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names: List,
        transforms: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """

        self.mode = mode
        self.transforms = transforms
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.img_path + self.image_names[idx]
        image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64')}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)

### Augs

In [7]:
sz = 576
sz_efffnetb4 = 600
deit_size = 384

In [8]:
TTA0_deit = albumentations.Compose([
            albumentations.Resize(deit_size, deit_size,p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA1_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.HorizontalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.VerticalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_5_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.Transpose(p=1.),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA3_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1
            ),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA4_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=1.
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA5_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=1.), 
            ToTensorV2()],
            p=1.)

TTA6_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.ShiftScaleRotate(p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA7_deit = albumentations.Compose([
            albumentations.CenterCrop(deit_size, deit_size),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=1.),
            ToTensorV2()],
            p=1.)

In [9]:
TTA0 = albumentations.Compose([
            albumentations.Resize(sz, sz,p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)



TTA1 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.HorizontalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.VerticalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_5 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.Transpose(p=1.),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA3 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1
            ),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA4 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=1.
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA5 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=1.), 
            ToTensorV2()],
            p=1.)

TTA6 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.ShiftScaleRotate(p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA7 = albumentations.Compose([
            albumentations.CenterCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=1.),
            ToTensorV2()],
            p=1.)

In [10]:
infer_augs_effnet = albumentations.Compose([
            albumentations.RandomResizedCrop(sz_efffnetb4, sz_efffnetb4),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)


infer_augs_resnext = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)

### PL module

In [11]:
def freeze_until(net: Any, param_name: str = None) -> None:
    """
    Freeze net until param_name


    Args:
        net:
        param_name:

    """
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

class BasicEncoder(nn.Module):
    def __init__(
        self,
        arch: str = 'resnet18',
        source: str = 'torchvision',
        pretrained: str = None,
        n_layers: int = -1,
        freeze: bool = False,
        to_one_channel: bool = False,
        freeze_until_layer: str = None,
    ) -> None:
        """
        Initialize Encoder.

        Args:
            num_classes: the number of target classes, the size of the last layer's output
            arch: the name of the architecture form pretrainedmodels
            pretrained: the mode for pretrained model from pretrainedmodels
            n_layers: number of layers to keep
            freeze: to freeze model
            freeze_until: freeze until this layer. If None, then freeze all layers
        """
        super().__init__()
        if source == 'timm':
            net = timm.create_model(arch, pretrained=pretrained)
            self.output_dimension = list(net.children())[-1].in_features
        else:
            print("Not implemented")
            
        if freeze:
            freeze_until(net, freeze_until_layer)
        
        layers = list(net.children())[:n_layers]    
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        output = self.layers(x)

        return output

In [12]:
class BasicDecoder(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Linear(output_dimension , n_classes)

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [13]:
class BasicDecoder_newhead(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output


In [14]:
class Net_B4_newhead(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [15]:
class Net_B3(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b3_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [16]:
class Net_B4(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [17]:
class Net_B5(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b5_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [18]:
class BasicDecoder_seresnext(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        #self.fc = nn.Linear(output_dimension , n_classes)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [19]:
class seResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='seresnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_seresnext(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [20]:
class ResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [21]:
class ResnextNet50d(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50d_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [22]:
class resnest50d(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnest50d_1s4x24d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [23]:
class DEIT(nn.Module):
    def __init__(self) -> None:
        
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_384', pretrained=True)
        output_dimension = list(self.encoder.children())[-1].in_features

        self.encoder.head = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
        self.loss = nn.CrossEntropyLoss()
    def forward(self, x, targets):
        logits = self.encoder(x)
        loss = self.loss(logits, targets).view(1)
        return logits, loss


In [24]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
        self.metric = pl.metrics.Accuracy()
        self.learning_rate = 1e-4

    def forward(self, x, targets, *args, **kwargs):
        return self.model(x, targets)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

        return (
            [optimizer],
            [{'scheduler': scheduler, 'interval': 'epoch', 'monitor': 'valid_loss'}],
        )

    def training_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'train_loss': loss, f'train_accuracy': score}
        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'train_accuracy': score,
        }

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)
        
        logs = {'train_loss': avg_loss, 'train_accuracy': score}
        return {'log': logs, 'progress_bar': logs}

    def validation_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'valid_loss': loss, f'valid_accuracy': score}

        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'valid_accuracy': score,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)

        # score = torch.tensor(1.0, device=self.device)
        logs = {'valid_loss': avg_loss, f'valid_accuracy': score, 'accuracy': score}
        return {'valid_loss': avg_loss, 'log': logs, 'progress_bar': logs}

In [25]:
labels.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [26]:
# path = "../input/cassava-leaf-disease-classification/"
# sub = pd.read_csv(f'{path}/sample_submission.csv')
# sub.head()

In [27]:
def getloader(df,path,bs=8,fold=0,tta=0,deit=False):
    
    if not deit:
        if tta == 0:
            augs = TTA0
        if tta == 1:
            augs = TTA1
        if tta == 2:
            augs = TTA2
        if tta == 3:
            augs = TTA2_5
        if tta == 4:
            augs = TTA3
        if tta == 5:
            augs = TTA4
        if tta == 6:
            augs = TTA5
        if tta == 7:
            augs = TTA6
        if tta == 8:
            augs = TTA7
    else:
        if tta == 0:
            augs = TTA0_deit
        if tta == 1:
            augs = TTA1_deit
        if tta == 2:
            augs = TTA2_deit
        if tta == 3:
            augs = TTA2_5_deit
        if tta == 4:
            augs = TTA3_deit
        if tta == 5:
            augs = TTA4_deit
        if tta == 6:
            augs = TTA5_deit
        if tta == 7:
            augs = TTA6_deit
        if tta == 8:
            augs = TTA7_deit
    
    print(f"Using TTA: {tta} and DEIT:{deit}")
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_indexes, valid_indexes = list(folds.split(df, df['label']))[fold]

    train_df = df.iloc[train_indexes]
    valid_df = df.iloc[valid_indexes]


    train_dataset = ImageClassificationDataset(image_names=train_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=train_df['label'].values,
                                                    img_path=path,
                                                    mode='train',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    valid_dataset = ImageClassificationDataset(image_names=valid_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=valid_df['label'].values,
                                                    img_path=path,
                                                    mode='valid',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=bs,num_workers=0,shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=bs,num_workers=0,shuffle=False)

    return train_loader,valid_loader,valid_indexes

In [28]:
# models = os.listdir('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss/')
# modelsPath = [os.path.join('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss',i) for i in models]
# modelsPath

In [29]:
from tqdm import tqdm
def getPreds(test_loader,lit_model,logitsOut=True):
    predictions = []
    
    for batch in tqdm(test_loader):
        image = batch['image'].to('cuda')
        target = batch['target'].to('cuda')
        with torch.no_grad():
            outputs = lit_model.model(image, target)[0]
            _outputs = outputs.softmax(1).detach().cpu().numpy()
            preds = outputs.argmax(1).detach().cpu().numpy()
            if not logitsOut:
                predictions.append(preds)
            else:
                #predictions[range(len(_outputs))] += _outputs
                predictions.append(_outputs)
    return np.array(predictions)

### Preds for Effnet b4 Distilled 2 SWA

In [30]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_4.ckpt'
             }

In [31]:
checkpoint = torch.load(modelsPath[0], map_location=lambda storage, loc: storage)
checkpoint.keys()

odict_keys(['model.encoder.layers.0.weight', 'model.encoder.layers.1.weight', 'model.encoder.layers.1.bias', 'model.encoder.layers.1.running_mean', 'model.encoder.layers.1.running_var', 'model.encoder.layers.1.num_batches_tracked', 'model.encoder.layers.3.0.0.conv_dw.weight', 'model.encoder.layers.3.0.0.bn1.weight', 'model.encoder.layers.3.0.0.bn1.bias', 'model.encoder.layers.3.0.0.bn1.running_mean', 'model.encoder.layers.3.0.0.bn1.running_var', 'model.encoder.layers.3.0.0.bn1.num_batches_tracked', 'model.encoder.layers.3.0.0.se.conv_reduce.weight', 'model.encoder.layers.3.0.0.se.conv_reduce.bias', 'model.encoder.layers.3.0.0.se.conv_expand.weight', 'model.encoder.layers.3.0.0.se.conv_expand.bias', 'model.encoder.layers.3.0.0.conv_pw.weight', 'model.encoder.layers.3.0.0.bn2.weight', 'model.encoder.layers.3.0.0.bn2.bias', 'model.encoder.layers.3.0.0.bn2.running_mean', 'model.encoder.layers.3.0.0.bn2.running_var', 'model.encoder.layers.3.0.0.bn2.num_batches_tracked', 'model.encoder.layer

In [32]:
preds_effnet_b4_distilled = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#     lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4_distilled[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.57it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:21<00:00,  1.64it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:21<00:00,  1.65it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:27<00:00,  1.53it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [33]:
preds_mean_effnet_b4_distilled = preds_effnet_b4_distilled.copy()
preds_mean_effnet_b4_distilled[:,0]= preds_effnet_b4_distilled[:,0]/(1+np.sqrt(0.05))
preds_mean_effnet_b4_distilled[:,1]= preds_effnet_b4_distilled[:,1]/(1+np.sqrt(0.102))
preds_mean_effnet_b4_distilled[:,2]= preds_effnet_b4_distilled[:,2]/(1+np.sqrt(0.111))
preds_mean_effnet_b4_distilled[:,3]= preds_effnet_b4_distilled[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnet_b4_distilled[:,4]= preds_effnet_b4_distilled[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnet_b4_distilled,1)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_distilled,1))

(0.8948450717390288, 0.8951254848810581)

In [35]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for DEIT

In [36]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_4.ckpt'}
             


In [37]:
preds_DEIT = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = DEIT()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta,deit=True)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_DEIT[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


Using cache found in C:\Users\Rajneesh Tiwari/.cache\torch\hub\facebookresearch_deit_main
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:True


100%|██████████| 134/134 [01:05<00:00,  2.03it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:True


...

100%|██████████| 134/134 [01:04<00:00,  2.07it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:True


100%|██████████| 134/134 [01:04<00:00,  2.08it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:True


100%|██████████| 134/134 [01:07<00:00,  1.98it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [38]:
preds_mean_DEIT = preds_DEIT.copy()
preds_mean_DEIT[:,0]= preds_DEIT[:,0]/(1+np.sqrt(0.05))
preds_mean_DEIT[:,1]= preds_DEIT[:,1]/(1+np.sqrt(0.102))
preds_mean_DEIT[:,2]= preds_DEIT[:,2]/(1+np.sqrt(0.111))
preds_mean_DEIT[:,3]= preds_DEIT[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_DEIT[:,4]= preds_DEIT[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_DEIT,1)

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_DEIT,1))

(0.8946113941206711, 0.8946581296443427)

In [40]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

340

### Preds for effnet B3

In [41]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'}
             


In [42]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [43]:
preds_effnet_b3 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B3()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b3[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:16<00:00,  1.75it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:14<00:00,  1.79it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:15<00:00,  1.78it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:21<00:00,  1.65it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [44]:
preds_mean_effnetb3 = preds_effnet_b3.copy()
preds_mean_effnetb3[:,0]= preds_effnet_b3[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb3[:,1]= preds_effnet_b3[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb3[:,2]= preds_effnet_b3[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb3[:,3]= preds_effnet_b3[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb3[:,4]= preds_effnet_b3[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb3,1)

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b3,1))

(0.8959199887834743, 0.896153666401832)

In [46]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

369

### Preds for seresnext

In [47]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [48]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [49]:
preds_seresnext = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = seResnextNet()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=128,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_seresnext[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 34/34 [01:41<00:00,  2.98s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 34/34 [01:39<00:00,  2.93s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 34/34 [01:38<00:00,  2.91s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 34/34 [01:45<00:00,  3.11s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [50]:
preds_mean_seresnext = preds_seresnext.copy()
preds_mean_seresnext[:,0]= preds_mean_seresnext[:,0]/(1+np.sqrt(0.05))
preds_mean_seresnext[:,1]= preds_mean_seresnext[:,1]/(1+np.sqrt(0.102))
preds_mean_seresnext[:,2]= preds_mean_seresnext[:,2]/(1+np.sqrt(0.111))
preds_mean_seresnext[:,3]= preds_mean_seresnext[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_seresnext[:,4]= preds_mean_seresnext[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_seresnext,1)

In [51]:
# classOut = np.argmax(preds_seresnext,1)
# classOut.shape

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_seresnext,1))

(0.8997055662008693, 0.8986773846800954)

In [53]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

360

In [54]:
torch.cuda.empty_cache()

In [55]:
gc.collect()

40

### Preds for Effnet b4 Taylor Loss - 2 SWA

In [56]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_4.ckpt'
             }

In [57]:
preds_effnet_b4_taylor = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4_taylor[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:23<00:00,  1.61it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:21<00:00,  1.64it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:21<00:00,  1.65it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:27<00:00,  1.52it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [58]:
preds_mean_effnetb4_taylor = preds_effnet_b4_taylor.copy()
preds_mean_effnetb4_taylor[:,0]= preds_effnet_b4_taylor[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4_taylor[:,1]= preds_effnet_b4_taylor[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4_taylor[:,2]= preds_effnet_b4_taylor[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4_taylor[:,3]= preds_effnet_b4_taylor[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4_taylor[:,4]= preds_effnet_b4_taylor[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4_taylor,1)

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_taylor,1))

(0.8965742861148759, 0.8969481703042482)

In [60]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for Effnet b4 Taylor Loss - 3 SWA

In [61]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [62]:
preds_effnet_b4_taylor_3SWA = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4_taylor_3SWA[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:23<00:00,  1.61it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:21<00:00,  1.64it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:21<00:00,  1.65it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:27<00:00,  1.53it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [63]:
preds_mean_effnetb4_taylor_3SWA = preds_effnet_b4_taylor_3SWA.copy()
preds_mean_effnetb4_taylor_3SWA[:,0]= preds_effnet_b4_taylor_3SWA[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4_taylor_3SWA[:,1]= preds_effnet_b4_taylor_3SWA[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4_taylor_3SWA[:,2]= preds_effnet_b4_taylor_3SWA[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4_taylor_3SWA[:,3]= preds_effnet_b4_taylor_3SWA[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4_taylor_3SWA[:,4]= preds_effnet_b4_taylor_3SWA[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4_taylor_3SWA,1)

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_taylor_3SWA,1))

(0.8970883768752629, 0.897181847922606)

In [65]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for B4 focal cosine loss

In [66]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [67]:
preds_effnet_b4 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:23<00:00,  1.61it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:21<00:00,  1.64it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:21<00:00,  1.65it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:27<00:00,  1.53it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [68]:
preds_mean_effnetb4 = preds_effnet_b4.copy()
preds_mean_effnetb4[:,0]= preds_effnet_b4[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4[:,1]= preds_effnet_b4[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4[:,2]= preds_effnet_b4[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4[:,3]= preds_effnet_b4[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4[:,4]= preds_effnet_b4[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4,1)

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4,1))

(0.8995653596298546, 0.8999859793428986)

In [70]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble b4 with deit

In [71]:
res_ = pd.DataFrame()
res_['b4_contrib'] = 0
#res['b5_contrib'] = 0
res_['deit_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    deit_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4+deit_contrib*preds_DEIT)
    res_.loc[count,'b4_contrib'] = i
    res_.loc[count,'deit_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [72]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,Accuracy
45,0.45,0.55,0.902136
54,0.54,0.46,0.901996
53,0.53,0.47,0.901996
55,0.55,0.45,0.901902
41,0.41,0.59,0.901855
...,...,...,...
4,0.04,0.96,0.895406
3,0.03,0.97,0.895266
2,0.02,0.98,0.895125
1,0.01,0.99,0.894985


### Ensemble 

In [73]:
res_ = pd.DataFrame()
res_['b4_contrib'] = 0
#res['b5_contrib'] = 0
res_['seresnext_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    seresnext_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext)
    res_.loc[count,'b4_contrib'] = i
    res_.loc[count,'seresnext_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [74]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,seresnext_contrib,Accuracy
16,0.16,0.84,0.902136
17,0.17,0.83,0.902042
15,0.15,0.85,0.901996
14,0.14,0.86,0.901996
8,0.08,0.92,0.901902
...,...,...,...
89,0.89,0.11,0.899939
88,0.88,0.12,0.899939
2,0.02,0.98,0.899893
1,0.01,0.99,0.899472


### Ensemble deit with b4 taylor loss

In [75]:
res_ = pd.DataFrame()
res_['b4_taylor_contrib'] = 0
#res['b5_contrib'] = 0
res_['deit_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    deit_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4_taylor+deit_contrib*preds_DEIT)
    res_.loc[count,'b4_taylor_contrib'] = i
    res_.loc[count,'deit_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [76]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_taylor_contrib,deit_contrib,Accuracy
49,0.49,0.51,0.901762
47,0.47,0.53,0.901668
48,0.48,0.52,0.901668
50,0.50,0.50,0.901435
46,0.46,0.54,0.901435
...,...,...,...
4,0.04,0.96,0.895780
3,0.03,0.97,0.895499
2,0.02,0.98,0.895266
1,0.01,0.99,0.894939


### Ensemble seresnext with b4 taylor loss

In [77]:
res_ = pd.DataFrame()
res_['b4_taylor_contrib'] = 0
#res['b5_contrib'] = 0
res_['seresnext_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    seresnext_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4_taylor+seresnext_contrib*preds_seresnext)
    res_.loc[count,'b4_taylor_contrib'] = i
    res_.loc[count,'seresnext_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [78]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_taylor_contrib,seresnext_contrib,Accuracy
12,0.12,0.88,0.901949
11,0.11,0.89,0.901855
14,0.14,0.86,0.901809
13,0.13,0.87,0.901809
18,0.18,0.82,0.901575
...,...,...,...
99,0.99,0.01,0.896995
95,0.95,0.05,0.896948
98,0.98,0.02,0.896948
96,0.96,0.04,0.896855


### Ensemble deit+seresnext+b4 focal cosine

In [79]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['deit_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        deit_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+deit_contrib*preds_mean_DEIT)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'deit_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [80]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1661,0.18,0.14,0.68,0.903117,0.902650
1906,0.21,0.16,0.63,0.903024,0.902697
1577,0.17,0.13,0.70,0.903024,0.902276
1826,0.20,0.16,0.64,0.902977,0.902603
1231,0.13,0.09,0.78,0.902977,0.902276
...,...,...,...,...,...
97,0.00,0.97,0.03,0.895079,0.894798
296,0.02,0.97,0.01,0.895079,0.894705
197,0.01,0.97,0.02,0.895079,0.894939
98,0.00,0.98,0.02,0.894985,0.894752


In [81]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1587,0.17,0.23,0.60,0.902136,0.903912
1502,0.16,0.22,0.62,0.902183,0.903818
1416,0.15,0.21,0.64,0.902276,0.903772
1752,0.19,0.23,0.58,0.902229,0.903772
1329,0.14,0.20,0.66,0.902183,0.903772
...,...,...,...,...,...
292,0.02,0.93,0.05,0.895312,0.894705
291,0.02,0.92,0.06,0.895359,0.894705
193,0.01,0.93,0.06,0.895125,0.894658
295,0.02,0.96,0.02,0.895125,0.894658


### Preds for B5

In [82]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [83]:
preds_effnet_b5 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B5()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b5[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:34<00:00,  1.41it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:33<00:00,  1.43it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:33<00:00,  1.44it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:39<00:00,  1.35it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [84]:
preds_mean_effnetb5 = preds_effnet_b5.copy()
preds_mean_effnetb5[:,0]= preds_effnet_b5[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb5[:,1]= preds_effnet_b5[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb5[:,2]= preds_effnet_b5[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb5[:,3]= preds_effnet_b5[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb5[:,4]= preds_effnet_b5[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb5,1)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b5,1))

(0.8965275505912044, 0.8982100294433799)

In [86]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble

In [87]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['b5_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        b5_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'b5_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [88]:
pd.set_option('display.max_rows',1000)

In [89]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1328,0.14,0.19,0.67,0.903117,0.902510
1327,0.14,0.18,0.68,0.903071,0.902556
1409,0.15,0.14,0.71,0.902977,0.902510
1325,0.14,0.16,0.70,0.902930,0.902603
1414,0.15,0.19,0.66,0.902930,0.902369
...,...,...,...,...,...
95,0.00,0.95,0.05,0.898397,0.896855
94,0.00,0.94,0.06,0.898397,0.896948
99,0.00,0.99,0.01,0.898350,0.896434
89,0.00,0.89,0.11,0.898350,0.897088


In [90]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1498,0.16,0.18,0.66,0.902603,0.902930
1497,0.16,0.17,0.67,0.902556,0.902884
1148,0.12,0.14,0.74,0.902416,0.902884
1149,0.12,0.15,0.73,0.902323,0.902884
1581,0.17,0.17,0.66,0.902603,0.902837
...,...,...,...,...,...
198,0.01,0.98,0.01,0.898537,0.896808
96,0.00,0.96,0.04,0.898397,0.896714
98,0.00,0.98,0.02,0.898444,0.896528
97,0.00,0.97,0.03,0.898584,0.896481


### resnest50d_1s4x24d

In [91]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [92]:
preds_resnest50d = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = resnest50d()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_resnest50d[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.57it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:23<00:00,  1.60it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:23<00:00,  1.60it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:30<00:00,  1.49it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [93]:
preds_mean_resnest50d = preds_resnest50d.copy()
preds_mean_resnest50d[:,0]= preds_resnest50d[:,0]/(1+np.sqrt(0.05))
preds_mean_resnest50d[:,1]= preds_resnest50d[:,1]/(1+np.sqrt(0.102))
preds_mean_resnest50d[:,2]= preds_resnest50d[:,2]/(1+np.sqrt(0.111))
preds_mean_resnest50d[:,3]= preds_resnest50d[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_resnest50d[:,4]= preds_resnest50d[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_resnest50d,1)

In [94]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_resnest50d,1))

(0.893816890218255, 0.897275318969949)

### Resnext50D

In [95]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [96]:
preds_resnext50d = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = ResnextNet50d()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
#     lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_resnext50d[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.41it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:34<00:00,  1.42it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:33<00:00,  1.43it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:39<00:00,  1.34it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [97]:
preds_mean_resnext50d = preds_resnext50d.copy()
preds_mean_resnext50d[:,0]= preds_resnext50d[:,0]/(1+np.sqrt(0.05))
preds_mean_resnext50d[:,1]= preds_resnext50d[:,1]/(1+np.sqrt(0.102))
preds_mean_resnext50d[:,2]= preds_resnext50d[:,2]/(1+np.sqrt(0.111))
preds_mean_resnext50d[:,3]= preds_resnext50d[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_resnext50d[:,4]= preds_resnext50d[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_resnext50d,1)

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_resnest50d,1))

(0.8905454035612469, 0.897275318969949)

### Ensemble

In [99]:
res_all4 = pd.DataFrame()
res_all4['b4_contrib'] = 0
res_all4['b5_contrib'] = 0
res_all4['seresnext_contrib'] = 0
res_all4['resnest_contrib'] = 0

res_all4 ['Accuracy'] = 0
res_all4 ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.05):
    for j in np.arange(0,1-i,0.05):
        for k in np.arange(0,1-i-j,0.05):
            b4_contrib = i
            b5_contrib = j
            seresnext_contrib = k
            resnest_contrib = 1-i-j-k
            print(b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib)
            #assert b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib==1.
            #print(i,j,k,1-(i+j+k))
            ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5+resnest_contrib*preds_mean_resnest50d)
            ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5+resnest_contrib*preds_resnest50d)
            res_all4.loc[count,'b4_contrib'] = i
            res_all4.loc[count,'seresnext_contrib'] = j
            res_all4.loc[count,'b5_contrib'] = k
            res_all4.loc[count,'resnest_contrib'] = 1-i-j-k
            res_all4.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
            res_all4.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
            count += 1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0

In [100]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,resnest_contrib,Accuracy,Adj_Accuracy
250,0.05,0.15,0.10,0.70,0.903444,0.902697
251,0.05,0.20,0.10,0.65,0.903398,0.902930
249,0.05,0.10,0.10,0.75,0.903351,0.902463
252,0.05,0.25,0.10,0.60,0.903257,0.902603
236,0.05,0.35,0.05,0.55,0.903117,0.902369
...,...,...,...,...,...,...
208,0.00,0.05,0.90,0.05,0.898537,0.896995
207,0.00,0.00,0.90,0.10,0.898490,0.896995
205,0.00,0.05,0.85,0.10,0.898444,0.897088
209,0.00,0.00,0.95,0.05,0.898444,0.896714


In [101]:
res_all4.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,resnest_contrib,Accuracy,Adj_Accuracy
476,0.10,0.50,0.20,0.20,0.902556,0.903024
477,0.10,0.55,0.20,0.15,0.902416,0.903024
632,0.15,0.60,0.15,0.10,0.902790,0.902977
474,0.10,0.40,0.20,0.30,0.902884,0.902977
251,0.05,0.20,0.10,0.65,0.903398,0.902930
...,...,...,...,...,...,...
204,0.00,0.00,0.85,0.15,0.898584,0.896948
209,0.00,0.00,0.95,0.05,0.898444,0.896714
2,0.00,0.10,0.00,0.90,0.900079,0.896481
1,0.00,0.05,0.00,0.95,0.899285,0.895593


In [102]:
#preds_DEIT
#preds_mean_DEIT

In [103]:
res_all4 = pd.DataFrame()
res_all4['DEIT_contrib'] = 0
res_all4['b5_contrib'] = 0
res_all4['seresnext_contrib'] = 0
res_all4['resnest_contrib'] = 0

res_all4 ['Accuracy'] = 0
res_all4 ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.05):
    for j in np.arange(0,1-i,0.05):
        for k in np.arange(0,1-i-j,0.05):
            DEIT_contrib = i
            b5_contrib = j 
            seresnext_contrib = k = 0 
            resnest_contrib = 1-i-j-k
            print(DEIT_contrib+b5_contrib+seresnext_contrib+resnest_contrib)
            #assert b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib==1.
            #print(i,j,k,1-(i+j+k))
            ens_preds_adj = (DEIT_contrib*preds_mean_DEIT+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5+resnest_contrib*preds_mean_resnest50d)
            ens_preds = (DEIT_contrib*preds_DEIT+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5+resnest_contrib*preds_resnest50d)
            res_all4.loc[count,'DEIT_contrib'] = i
            res_all4.loc[count,'seresnext_contrib'] = j
            res_all4.loc[count,'b5_contrib'] = k
            res_all4.loc[count,'resnest_contrib'] = 1-i-j-k
            res_all4.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
            res_all4.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
            count += 1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.999999

In [104]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_effnet_b4,1))

array([[1.        , 0.54561218],
       [0.54561218, 1.        ]])

In [105]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_effnet_b5,1))

array([[1.        , 0.64003239],
       [0.64003239, 1.        ]])

In [106]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_seresnext,1))

array([[1.        , 0.57645585],
       [0.57645585, 1.        ]])

In [107]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_resnest50d,1))

array([[1.        , 0.49928238],
       [0.49928238, 1.        ]])

In [108]:
np.corrcoef(np.max(preds_seresnext,1),np.max(preds_resnest50d,1))

array([[1.        , 0.89773932],
       [0.89773932, 1.        ]])

In [109]:
np.corrcoef(np.max(preds_effnet_b4_taylor,1),np.max(preds_effnet_b4,1))

array([[1.       , 0.6273762],
       [0.6273762, 1.       ]])

In [110]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

,DEIT_contrib,b5_contrib,seresnext_contrib,resnest_contrib,Accuracy,Adj_Accuracy
1062,0.30,0.0,0.35,0.35,0.903772,0.902136
1063,0.30,0.0,0.35,0.35,0.903772,0.902136
1064,0.30,0.0,0.35,0.35,0.903772,0.902136
1065,0.30,0.0,0.35,0.35,0.903772,0.902136
1066,0.30,0.0,0.35,0.35,0.903772,0.902136
...,...,...,...,...,...,...
1546,0.90,0.0,0.00,0.10,0.895125,0.894939
1548,0.95,0.0,0.00,0.05,0.895079,0.894845
1540,0.85,0.0,0.00,0.15,0.895032,0.894892
1541,0.85,0.0,0.00,0.15,0.895032,0.894892


In [111]:
from sklearn.metrics import confusion_matrix

confusion_matrix(labels.label,np.argmax(preds_effnet_b4,1),normalize='true')

array([[0.65133395, 0.07267709, 0.02575897, 0.04783809, 0.2023919 ],
       [0.04065783, 0.82594792, 0.02603929, 0.03837369, 0.06898127],
       [0.01047779, 0.02053646, 0.82564962, 0.09555742, 0.04777871],
       [0.00174799, 0.00402797, 0.01360389, 0.9746922 , 0.00592795],
       [0.06713232, 0.04772992, 0.05122235, 0.07877377, 0.75514164]])

In [112]:
confusion_matrix(labels.label,np.argmax(preds_mean_effnetb4,1),normalize='true')

array([[0.65961362, 0.07083717, 0.02575897, 0.04691812, 0.19687213],
       [0.04339881, 0.8241206 , 0.02603929, 0.03791686, 0.06852444],
       [0.01131601, 0.02095557, 0.82900251, 0.09052808, 0.04819782],
       [0.00189998, 0.00425597, 0.01459188, 0.97309622, 0.00615595],
       [0.06868452, 0.04772992, 0.05199845, 0.07683353, 0.75475359]])

In [113]:
confusion_matrix(labels.label,np.argmax(preds_effnet_b4_taylor,1),normalize='true')

array([[0.65409384, 0.0625575 , 0.02391904, 0.03955842, 0.21987121],
       [0.04294198, 0.80995889, 0.02832344, 0.04157149, 0.0772042 ],
       [0.01131601, 0.01760268, 0.8319363 , 0.09094719, 0.04819782],
       [0.00212798, 0.00395197, 0.01573187, 0.97134823, 0.00683995],
       [0.06829647, 0.04772992, 0.05549088, 0.07489329, 0.75358945]])

In [114]:
confusion_matrix(labels.label,np.argmax(preds_mean_effnetb4_taylor,1),normalize='true')

array([[0.65869365, 0.06347746, 0.02299908, 0.03679853, 0.21803128],
       [0.04431247, 0.81224303, 0.02832344, 0.03883052, 0.07629054],
       [0.01173512, 0.01760268, 0.83487008, 0.0875943 , 0.04819782],
       [0.00220398, 0.00440796, 0.01656787, 0.96929625, 0.00752394],
       [0.06946061, 0.04772992, 0.05626698, 0.07217695, 0.75436554]])

In [115]:
confusion_matrix(labels.label,np.argmax(preds_DEIT,1),normalize='true')

array([[0.66697332, 0.07451702, 0.01931923, 0.04231831, 0.19687213],
       [0.05070809, 0.80310644, 0.02512563, 0.04339881, 0.07766103],
       [0.01383068, 0.02305113, 0.79715004, 0.10813076, 0.05783738],
       [0.00197598, 0.00448396, 0.01147591, 0.97568019, 0.00638395],
       [0.07450524, 0.05122235, 0.04656577, 0.08265425, 0.74505239]])

In [116]:
confusion_matrix(labels.label,np.argmax(preds_effnet_b5,1),normalize='true')

array([[0.68629255, 0.06071757, 0.02391904, 0.04231831, 0.18675253],
       [0.04248515, 0.81041571, 0.03197807, 0.03837369, 0.07674737],
       [0.01131601, 0.01760268, 0.83361274, 0.08717519, 0.05029338],
       [0.00227998, 0.00448396, 0.01557988, 0.96990424, 0.00775194],
       [0.07062476, 0.04811797, 0.05549088, 0.06984866, 0.75591773]])

In [117]:
confusion_matrix(labels.label,np.argmax(preds_resnest50d,1),normalize='true')

array([[0.66145354, 0.06531739, 0.02207912, 0.04783809, 0.20331187],
       [0.04431247, 0.81772499, 0.0246688 , 0.04065783, 0.07263591],
       [0.01173512, 0.01844091, 0.81223806, 0.10352054, 0.05406538],
       [0.00197598, 0.00349597, 0.0128439 , 0.9753002 , 0.00638395],
       [0.07217695, 0.04889406, 0.05393869, 0.08032596, 0.74466434]])

In [118]:
confusion_matrix(labels.label,np.argmax(preds_resnext50d,1),normalize='true')

array([[0.66513339, 0.05519779, 0.02023919, 0.04783809, 0.21159154],
       [0.04659662, 0.80950206, 0.02284148, 0.04294198, 0.07811786],
       [0.01257334, 0.01802179, 0.80846605, 0.10100587, 0.05993294],
       [0.00197598, 0.00402797, 0.01367989, 0.97256422, 0.00775194],
       [0.07140085, 0.04617773, 0.0508343 , 0.07916182, 0.7524253 ]])

In [119]:
confusion_matrix(labels.label,np.argmax(preds_seresnext,1),normalize='true')

array([[0.67341306, 0.06439742, 0.02115915, 0.04875805, 0.19227231],
       [0.04294198, 0.80721791, 0.02695295, 0.04431247, 0.07857469],
       [0.01173512, 0.02095557, 0.80511316, 0.11190277, 0.05029338],
       [0.00212798, 0.00334397, 0.01086791, 0.97811218, 0.00554796],
       [0.07217695, 0.04578968, 0.04772992, 0.08187815, 0.7524253 ]])

In [120]:
b4_contrib = 0.18
seresnext_contrib = 0.61
b5_contrib = 0.21
_ = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

0.9026499041921765


array([[0.68353266, 0.06163753, 0.02023919, 0.04599816, 0.18859246],
       [0.04385564, 0.82000914, 0.02786661, 0.03928735, 0.06898127],
       [0.01131601, 0.01844091, 0.82187762, 0.09891031, 0.04945516],
       [0.00182399, 0.00364797, 0.0122359 , 0.97682019, 0.00547196],
       [0.06558013, 0.04656577, 0.04928211, 0.07722158, 0.76135041]])

In [121]:
deit_contrib = 0.15
seresnext_contrib = 0.25
resnest_contrib = 0.6
_ = (deit_contrib*preds_DEIT+seresnext_contrib*preds_seresnext+resnest_contrib*preds_resnest50d)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

0.8996120951535262


array([[0.67617295, 0.07083717, 0.01655934, 0.04415823, 0.19227231],
       [0.04705345, 0.81544084, 0.02375514, 0.04294198, 0.07080859],
       [0.01257334, 0.0217938 , 0.80134116, 0.10980721, 0.05448449],
       [0.00174799, 0.00364797, 0.00987992, 0.97925217, 0.00547196],
       [0.06984866, 0.05122235, 0.04617773, 0.0830423 , 0.74970896]])

In [122]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['deit_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        deit_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+deit_contrib*preds_mean_DEIT)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'deit_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [123]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1661,0.18,0.14,0.68,0.903117,0.902650
1906,0.21,0.16,0.63,0.903024,0.902697
1577,0.17,0.13,0.70,0.903024,0.902276
1826,0.20,0.16,0.64,0.902977,0.902603
1231,0.13,0.09,0.78,0.902977,0.902276
...,...,...,...,...,...
97,0.00,0.97,0.03,0.895079,0.894798
296,0.02,0.97,0.01,0.895079,0.894705
197,0.01,0.97,0.02,0.895079,0.894939
98,0.00,0.98,0.02,0.894985,0.894752


In [124]:
labels.label.value_counts()

3    13158
4     2577
2     2386
1     2189
0     1087
Name: label, dtype: int64

### Below looks good

In [125]:
deit_contrib = 0.25
seresnext_contrib = 0.46
b4_contrib = 0.29
_ = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

0.9023694910501472


array([[0.67157314, 0.06899724, 0.02115915, 0.04599816, 0.19227231],
       [0.04157149, 0.82549109, 0.02284148, 0.04294198, 0.06715395],
       [0.01173512, 0.02011735, 0.81726739, 0.10268231, 0.04819782],
       [0.00174799, 0.00357197, 0.01117191, 0.97826417, 0.00524396],
       [0.06790842, 0.04656577, 0.04889406, 0.08032596, 0.75630578]])